# PCE Calculations of Testing Dataset

In [ ]:
import ASCVD_Calc_PCE
import pandas as pd
import numpy as np
from importlib import reload
reload(ASCVD_Calc_PCE)
from sklearn.metrics import roc_auc_score, roc_curve

## Import Data

In [2]:
df = pd.read_csv('/Volumes/fsmresfiles/PrevMed/Projects/MESA_RiskPred/LRPP data/LRPP_select.csv')
indecies_test = pd.read_csv('/Volumes/fsmresfiles/PrevMed/Projects/MESA_RiskPred/LRPP data/mesa_index_select.csv')
indecies_test = indecies_test.loc[indecies_test['study_index'] == True, :]
df = df.loc[df['id'].isin(indecies_test['id']), :]

In [3]:
# filter out patients without data before 8 years
max_age_before_threshold = df.loc[df['times'] <= 8, :].groupby('id').times.max().reset_index()
df = df.loc[df['id'].isin(max_age_before_threshold['id']), :]

# get labels and tte
te_label = np.array(df.loc[:,['id', 'label']].drop_duplicates().label)
te_time = np.array(df.loc[:, ['id', 'tte']].drop_duplicates().tte)

## PCE Calculation

In [4]:
# get risk measurements from PCE data set (Multiple)
pce_pred_df, pce_df = ASCVD_Calc_PCE.pce_pred_df_tab(df, 8, 10)

In [5]:
# calculate AUC 
def prediction_auc_PCE_df(pred_time, pred_time_index, pce_pred_df, cluster):
    """
    Calculates the AUC of the PCE predictions at a given time
    pred_time: pred time list of prediction times
    pred_time_index: which prediction time in terms of position in the list 
    pce_pred_df: df of probability of PCE dataset
    """
    global true, time_horizon
    
    time_horizon = pred_time + 10 
    true = (te_time <= time_horizon) * (te_label == 1).astype(int)
    
    pce_pred_df['true_label'] = true
    pce_pred_df = pce_pred_df.loc[~pce_pred_df.risk.isnull(),:]
    
    if cluster != 'none':
        pce_pred_df = pce_pred_df.loc[pce_pred_df['labels'] == cluster, :]
        
    return pce_pred_df

def prediction_auc_PCE(pred_time, pred_time_index, pce_pred_df, cluster):
    
    pce_pred_df = prediction_auc_PCE_df(pred_time, pred_time_index, pce_pred_df, cluster)
    auc = roc_auc_score(pce_pred_df['true_label'].tolist(), pce_pred_df['risk'].tolist())
    
    return auc 

def prediction_ROC(pred_time, pred_time_index, pce_pred_df, cluster):
    
    pce_pred_df = prediction_auc_PCE_df(pred_time, pred_time_index, pce_pred_df, cluster)
    fpr, tpr, thresh = roc_curve(pce_pred_df['true_label'].tolist(), pce_pred_df['risk'].tolist())
    
    return fpr, tpr, thresh

In [6]:
# Get PCE AUC for Each Prediction Year
print('NULL Risk Values from PCE: ', pce_pred_df.risk.isna().sum())
print('PCE AUC:', np.round(prediction_auc_PCE(8, 10, pce_pred_df, 'none'),3))

NULL Risk Values from PCE:  0
PCE AUC: 0.741
